# Aive Case Study

Cells are meant to be run in order. During the output of either video (face or human), press enter to stop and end the analysis. Sometimes it's buggy and you need to go Kernel -> Restart & Clear Output to kill the video popup.

Install packages manually in a venv, or used the anaconda3 kernel for the notebook for best results. The YouTube video needs to be downloaded as an mp4 in this case as well.

I use OpenCV, which seems pretty standard, and tune paramters based on some initial high level advice here: https://www.pyimagesearch.com/2015/11/16/hog-detectmultiscale-parameters-explained/

## Imports and global vars

In [ ]:
import numpy as np
import cv2
import face_recognition

In [ ]:
cap = cv2.VideoCapture("MISS DIOR The new Eau de Parfum.mp4")

# Face detection

Borrowed heavily from: https://www.analyticsvidhya.com/blog/2018/12/introduction-face-detection-video-deep-learning-python/

In [ ]:
face_locations = []

while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    
    # Convert the image from BGR color (which OpenCV uses) to RGB   
    # color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]
    
    # Find all the faces in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    
    for top, right, bottom, left in face_locations:
        # Draw a box around the face
        cv2.rectangle(frame, 
                      (left, top), 
                      (right, bottom), 
                      (0, 0, 255), 
                      2)
        
    # Display the resulting image
    cv2.imshow('Video-Faces', 
               frame)

    # Wait for Enter key to stop
    if cv2.waitKey(25) == 13:
        break

cap.release()
cv2.destroyAllWindows()

## Human detection

Borrowed heavily from: https://thedatafrog.com/en/articles/human-detection-video/#:~:text=OpenCV%20features%20an%20implementation%20for,work%20well%20in%20other%20cases.

In [ ]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    try:
        # resizing for faster detection
        frame = cv2.resize(frame, 
                           (640, 480))
        # using a greyscale picture, also for faster detection
        gray = cv2.cvtColor(frame, 
                            cv2.COLOR_RGB2GRAY)

        # detect people in the image
        # returns the bounding boxes for the detected objects
        boxes, weights = hog.detectMultiScale(frame, 
                                              winStride=(4,4),
                                              scale=1.01)
        boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

        for (xA, yA, xB, yB) in boxes:
            # display the detected boxes in the colour picture
            cv2.rectangle(frame, 
                          (xA, yA), 
                          (xB, yB),
                          (0, 255, 0), 
                          2)

        # Display the resulting frame
        cv2.imshow('Video-Humans',
                   frame)
        if cv2.waitKey(25) == 13:
            break
            
    except:
        #print('Bad Frame')
        pass

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## Next Steps

The face identifier works better out of the box with minimal tuning than the huamn identifier, but both still need a lot of work. It's interesting that in so few lines of code, we can get a MVP up and running, but as a next step, I'd:

* Tune the hypermarameters better;
* Implement https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
* Train the model on similar videos and pictures